In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import time

# --- 표정 이미지 로드 및 체크 ---
# PNG(알파채널 포함)로 표정 이미지를 불러온다. 각 이미지는 투명 배경이어야 한다.
face_default = cv2.imread('face_default.png', cv2.IMREAD_UNCHANGED)  # 기본 얼굴
face_cry = cv2.imread('face_cry.png', cv2.IMREAD_UNCHANGED)        # 우는 얼굴
face_strain = cv2.imread('face_strain.png', cv2.IMREAD_UNCHANGED)  # 찡그린 얼굴
face_zzz = cv2.imread('face_zzz.png', cv2.IMREAD_UNCHANGED)        # 졸린 얼굴

def check_image(img, name):
    """이미지 로드 및 알파채널(4채널) 여부 확인"""
    if img is None:
        print(f"{name} 이미지를 불러오지 못했습니다.")
        return False
    if len(img.shape) != 3 or img.shape[2] != 4:
        print(f"{name} 이미지 shape 오류: {img.shape}")
        return False
    return True

# 각 표정 이미지가 정상적으로 불러와졌는지 확인
assert check_image(face_cry, "face_cry")
assert check_image(face_default, "face_default")
assert check_image(face_strain, "face_strain")
assert check_image(face_zzz, "face_zzz")

# --- Skeleton(관절 연결선) 색상 지정 ---
# 주요 관절 연결선에 색상을 지정하여 시각적으로 구분한다.
skeleton_colors = {
    ('left_wrist', 'left_elbow'): (0, 0, 255),   # 왼손목-왼팔꿈치: 빨강
    ('right_wrist', 'right_elbow'): (255, 0, 0), # 오른손목-오른팔꿈치: 파랑
    # ... (다른 부위도 추가 가능)
}

# --- Keypoint 이름과 인덱스 매핑 ---
# YOLOv8 pose keypoint 순서에 맞춰 이름-인덱스 매핑 딕셔너리 생성
keypoint_name_to_index = {
    'nose': 0, 'left_eye': 1, 'right_eye': 2, 'left_ear': 3, 'right_ear': 4,
    'left_shoulder': 5, 'right_shoulder': 6, 'left_elbow': 7, 'right_elbow': 8,
    'left_wrist': 9, 'right_wrist': 10, 'left_hip': 11, 'right_hip': 12,
    'left_knee': 13, 'right_knee': 14, 'left_ankle': 15, 'right_ankle': 16
}

def get_face_rect(face_pts):
    """얼굴 keypoints(코, 눈, 귀)로 얼굴 bounding box 계산 및 패딩 적용"""
    x_min = np.min(face_pts[:,0])
    y_min = np.min(face_pts[:,1])
    x_max = np.max(face_pts[:,0])
    y_max = np.max(face_pts[:,1])
    w = x_max - x_min
    h = y_max - y_min
    pad = int(0.5 * max(w, h))  # 얼굴 영역을 넉넉하게 잡기 위해 패딩 적용
    x_min = max(x_min - pad, 0)
    y_min = max(y_min - pad, 0)
    w = w + 2*pad
    h = h + 2*pad
    return x_min, y_min, w, h

def overlay_image(bg, fg, rect):
    """배경(bg)에 전경(fg) 이미지를 rect 위치/크기로 알파블렌딩 오버레이"""
    x, y, w, h = [int(v) for v in rect]
    orig_h, orig_w = fg.shape[:2]
    aspect = orig_w / orig_h
    # 얼굴 영역 비율에 맞게 전경 이미지를 리사이즈 (비율 유지)
    if w / h > aspect:
        new_h = h
        new_w = int(h * aspect)
    else:
        new_w = w
        new_h = int(w / aspect)
    fg_resized = cv2.resize(fg, (new_w, new_h))
    x_offset = x + (w - new_w) // 2
    y_offset = y + (h - new_h) // 2

    # 프레임 경계 클리핑 (오버레이 영역이 프레임 밖으로 나가지 않게)
    x1 = max(x_offset, 0)
    y1 = max(y_offset, 0)
    x2 = min(x_offset + new_w, bg.shape[1])
    y2 = min(y_offset + new_h, bg.shape[0])

    fg_x1 = max(0, -x_offset)
    fg_y1 = max(0, -y_offset)
    fg_x2 = fg_x1 + (x2 - x1)
    fg_y2 = fg_y1 + (y2 - y1)

    # 알파 채널을 이용한 블렌딩
    if fg_resized.shape[2] == 4:
        alpha = fg_resized[fg_y1:fg_y2, fg_x1:fg_x2, 3] / 255.0
        for c in range(3):
            bg[y1:y2, x1:x2, c] = (
                (1 - alpha) * bg[y1:y2, x1:x2, c] + alpha * fg_resized[fg_y1:fg_y2, fg_x1:fg_x2, c]
            )
    else:
        bg[y1:y2, x1:x2] = fg_resized[fg_y1:fg_y2, fg_x1:fg_x2]
    return bg

# YOLOv8 pose 모델 로드
model = YOLO('yolov8n-pose.pt')  # B 단계
cap = cv2.VideoCapture(0)        # A 단계

# 각 사람(인덱스별)마다 손/얼굴 상태, 움직임, palm 방향 등을 저장
person_states = {}  # C 단계

def palm_direction(wrist, elbow):
    """손바닥 방향 추정: wrist->elbow 벡터로 위/옆/기타 판별"""
    vec = elbow - wrist
    if vec[1] < -20:
        return 'up'
    elif abs(vec[0]) > abs(vec[1]):
        return 'side'
    else:
        return 'other'

def is_hand_up(kps):
    """한 손이라도 코보다 위에 있으면 True 반환"""
    nose_y = kps[0][1]
    left_wrist_y = kps[9][1]
    right_wrist_y = kps[10][1]
    return left_wrist_y < nose_y or right_wrist_y < nose_y

# --- 메인 루프: 프레임별 처리 ---
while True:
    try:
        # A: 웹캠에서 프레임을 읽어옴
        ret, frame = cap.read()
        if not ret:
            print("웹캠 프레임을 읽지 못했습니다.")
            break

        frame = cv2.resize(frame, (640, 480))  # 처리 속도/일관성 위해 프레임 크기 고정
        # B: YOLOv8 pose 모델로 사람/관절 추론
        results = model(frame, stream=True)    # YOLOv8 pose 추론 (스트림 모드)
        for r in results:
            # C: 각 사람의 keypoints, bounding box 추출
            boxes = r.boxes.xyxy.cpu().numpy().astype(int)      # 각 사람 bounding box
            keypoints = r.keypoints.xy.cpu().numpy().astype(int) # 각 사람 keypoints
            for i, kps in enumerate(keypoints):
                if kps.shape[0] < 11:
                    print("keypoints shape 오류:", kps.shape)
                    continue
                # C: Bounding box 시각화
                x1, y1, x2, y2 = boxes[i]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                # D: 얼굴 위치/크기 추정 및 유효성 체크
                face_pts = kps[[0,1,2,3,4]]
                fx, fy, fw, fh = get_face_rect(face_pts)
                valid_face = True
                for pt in face_pts:
                    if pt[0] < 0 or pt[1] < 0 or pt[0] >= frame.shape[1] or pt[1] >= frame.shape[0]:
                        valid_face = False
                        break
                # K: 얼굴 영역이 작거나 유효하지 않으면 해당 사람은 스킵
                if not valid_face or fw <= 60 or fh <= 60 or (x2-x1) <= 80 or (y2-y1) <= 80:
                    continue
                # E: 손 위치 판단 및 움직임 감지, 표정 결정
                left_wrist = kps[9]
                right_wrist = kps[10]
                left_elbow = kps[7]
                right_elbow = kps[8]
                wrists = np.array([left_wrist, right_wrist])
                if i not in person_states:
                    person_states[i] = {
                        'prev_wrist': None,
                        'face_default_time': None,
                        'face_zzz_active': False,
                        'prev_left_palm': None,
                        'prev_right_palm': None
                    }
                state = person_states[i]
                move_dist = 0
                if state['prev_wrist'] is not None:
                    move_dist = np.linalg.norm(wrists - state['prev_wrist'])
                    if move_dist > 20:
                        state['face_default_time'] = None
                        state['face_zzz_active'] = False
                state['prev_wrist'] = wrists
                left_palm = palm_direction(left_wrist, left_elbow)
                right_palm = palm_direction(right_wrist, right_elbow)
                # D: 얼굴이 프레임에서 아주 크게 감지되면 face_default 표시
                frame_h, frame_w = frame.shape[:2]
                if fw > frame_w * 0.5 or fh > frame_h * 0.5:
                    face_img = face_default
                    state['face_default_time'] = None
                    state['face_zzz_active'] = False
                # E: 표정 결정 로직
                elif is_hand_up(kps):
                    face_img = face_cry  # 한 손이라도 코보다 위면 우는 표정
                    state['face_default_time'] = None
                    state['face_zzz_active'] = False
                elif not is_hand_up(kps):
                    face_img = face_zzz  # 손을 내리면 졸린 얼굴
                    state['face_default_time'] = None
                    state['face_zzz_active'] = True
                elif state['face_zzz_active']:
                    face_img = face_zzz
                    if move_dist > 10 or left_palm != state['prev_left_palm'] or right_palm != state['prev_right_palm']:
                        state['face_zzz_active'] = False
                        face_img = face_default
                        state['face_default_time'] = time.time()
                else:
                    if left_palm == 'up' or right_palm == 'up':
                        face_img = face_cry
                        state['face_default_time'] = None
                    elif left_palm == 'side' or right_palm == 'side':
                        face_img = face_strain
                        state['face_default_time'] = None
                    else:
                        face_img = face_default
                        if state['face_default_time'] is None:
                            state['face_default_time'] = time.time()
                        elif time.time() - state['face_default_time'] > 3:
                            face_img = face_zzz
                            state['face_zzz_active'] = True
                state['prev_left_palm'] = left_palm
                state['prev_right_palm'] = right_palm
                # F: 결정된 표정에 맞는 얼굴 이미지 선택 및 오버레이
                frame = overlay_image(frame, face_img, (fx, fy, fw, fh))
                # H: skeleton(관절 연결선) 시각화
                for (p1, p2), color in skeleton_colors.items():
                    idx1, idx2 = keypoint_name_to_index[p1], keypoint_name_to_index[p2]
                    cv2.line(frame, tuple(kps[idx1]), tuple(kps[idx2]), color, 3)
        # J: 결과 프레임을 화면에 출력
        cv2.imshow('result preview', frame)  # 결과 프레임 출력
        if cv2.waitKey(33) == ord('q'):      # 'q' 입력 시 종료
            break
    except Exception as e:
        print("오류 발생:", e)
        cv2.waitKey(0)  # 에러 발생 시 창이 바로 닫히지 않게 대기
        break
cap.release()
cv2.destroyAllWindows()


0: 480x640 2 persons, 54.0ms
Speed: 1.8ms preprocess, 54.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 2 persons, 54.0ms
Speed: 1.8ms preprocess, 54.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 2 persons, 6.5ms
Speed: 1.4ms preprocess, 6.5ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 2 persons, 6.5ms
Speed: 1.4ms preprocess, 6.5ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 6.0ms

0: 480x640 2 persons, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Speed: 1.5ms preprocess, 6.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 6.5ms

0: 480x640 2 persons, 6.5ms
Speed: 1.1ms preprocess, 6.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Speed: 1.1ms preprocess, 6.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 64

# Emoticon Project 전체 처리 흐름도 (Mermaid)

```mermaid
flowchart TD
    A[웹캠 프레임 캡처] --> B[YOLOv8 Pose 추론]
    B --> C[사람별 Keypoints, Bounding Box 추출]
    C --> D{얼굴 영역 유효성/크기 체크}
    D -- 유효/충분히 큼 --> E[표정 결정 로직]
    D -- 유효하지 않음/작음 --> K[스킵]
    E --> F[얼굴 대체 이미지 선택]
    F --> G[얼굴 이미지 오버레이]
    C --> H[관절 연결선 시각화]
    G --> I[프레임 합성]
    H --> I
    I --> J[결과 프레임 출력 (cv2.imshow)]
    J --> A
```

- **A**: 웹캠에서 프레임을 읽어옴
- **B**: YOLOv8 pose 모델로 사람/관절 추론
- **C**: 각 사람의 keypoints, bounding box 추출
- **D**: 얼굴 keypoint가 프레임 내에 있고, 얼굴 영역이 충분히 큰지 체크
- **E**: 손 위치, palm 방향, 움직임 등으로 표정 결정
- **F**: 결정된 표정에 맞는 얼굴 이미지 선택
- **G**: 얼굴 영역에 이미지 오버레이
- **H**: skeleton(관절 연결선) 시각화
- **I**: 오버레이/스켈레톤이 적용된 프레임 합성
- **J**: 결과 프레임을 화면에 출력
- **K**: 얼굴 영역이 작거나 유효하지 않으면 해당 사람은 스킵